<a href="https://colab.research.google.com/github/HanjunHeatherLi/Clinical_no_show_project/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas_profiling import ProfileReport 
jupyter nbextension enable --py widgetsnbextension

SyntaxError: ignored

In [2]:
workingpath = "/content/drive/MyDrive/Colab Notebooks/project/" 

#Data loading section

In [3]:
df=pd.read_csv( workingpath + "KaggleV2_May_2016.csv")
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589980e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262960e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679510e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841190e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


# data clean up

##check duplicate and na

In [4]:
df.duplicated().any()

False

In [5]:
df.isnull().values.any()


False

In [6]:
df[df.Age==-1].shape[0]

1

In [7]:
df['Handcap'].value_counts()

0    108286
1      2042
2       183
3        13
4         3
Name: Handcap, dtype: int64

In [8]:
df = df[df.Age >= 0]

In [9]:
df.shape

(110526, 14)

##remove patient&appointment ID


In [10]:
df.drop(['PatientId','AppointmentID'],axis=1,inplace=True)


In [11]:
df.AppointmentDay=pd.to_datetime(df.AppointmentDay)
df.ScheduledDay=pd.to_datetime(df.ScheduledDay)
df['daystoappoint']=(df.AppointmentDay-df.ScheduledDay).dt.days+1
df.drop(['ScheduledDay','AppointmentDay'],axis=1,inplace=True)


##make dummy variables

###gender

In [14]:
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender']) # 0 is famale 1 is male

###Neighbourhood

In [15]:
n = len(pd.unique(df['Neighbourhood']))
n

81

In [16]:
dummy=pd.get_dummies(df.Neighbourhood, prefix='NB')
df=df.merge(dummy, how='outer', left_index=True, right_index=True)
df.drop(['Neighbourhood'],axis=1,inplace=True)


In [17]:
dummy=pd.get_dummies(df.Handcap, prefix='Hcap')
df=df.merge(dummy, how='outer', left_index=True, right_index=True)
df.drop(['Handcap'],axis=1,inplace=True)


,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,daystoappoint,NB_AEROPORTO,NB_ANDORINHAS,NB_ANTÔNIO HONÓRIO,NB_ARIOVALDO FAVALESSA,NB_BARRO VERMELHO,NB_BELA VISTA,NB_BENTO FERREIRA,NB_BOA VISTA,NB_BONFIM,NB_CARATOÍRA,NB_CENTRO,NB_COMDUSA,NB_CONQUISTA,NB_CONSOLAÇÃO,NB_CRUZAMENTO,NB_DA PENHA,NB_DE LOURDES,NB_DO CABRAL,NB_DO MOSCOSO,NB_DO QUADRO,NB_ENSEADA DO SUÁ,NB_ESTRELINHA,NB_FONTE GRANDE,NB_FORTE SÃO JOÃO,NB_FRADINHOS,NB_GOIABEIRAS,NB_GRANDE VITÓRIA,NB_GURIGICA,NB_HORTO,NB_ILHA DAS CAIEIRAS,NB_ILHA DE SANTA MARIA,...,NB_MONTE BELO,NB_MORADA DE CAMBURI,NB_MÁRIO CYPRESTE,NB_NAZARETH,NB_NOVA PALESTINA,NB_PARQUE INDUSTRIAL,NB_PARQUE MOSCOSO,NB_PIEDADE,NB_PONTAL DE CAMBURI,NB_PRAIA DO CANTO,NB_PRAIA DO SUÁ,NB_REDENÇÃO,NB_REPÚBLICA,NB_RESISTÊNCIA,NB_ROMÃO,NB_SANTA CECÍLIA,NB_SANTA CLARA,NB_SANTA HELENA,NB_SANTA LUÍZA,NB_SANTA LÚCIA,NB_SANTA MARTHA,NB_SANTA TEREZA,NB_SANTO ANDRÉ,NB_SANTO ANTÔNIO,NB_SANTOS DUMONT,NB_SANTOS REIS,NB_SEGURANÇA DO LAR,NB_SOLON BORGES,NB_SÃO BENEDITO,NB_SÃO CRISTÓVÃO,NB_SÃO JOSÉ,NB_SÃO PEDRO,NB_TABUAZEIRO,NB_UNIVERSITÁRIO,NB_VILA RUBIM,Hcap_0,Hcap_1,Hcap_2,Hcap_3,Hcap_4
0,0,62,0,1,0,0,0,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,56,0,0,0,0,0,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,62,0,0,0,0,0,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,8,0,0,0,0,0,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,56,0,1,1,0,0,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


###dependent variable No_show to 0 or 1

In [22]:
df['No-show'] = le.fit_transform(df['No-show'])
df.head()

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,daystoappoint,NB_AEROPORTO,NB_ANDORINHAS,NB_ANTÔNIO HONÓRIO,NB_ARIOVALDO FAVALESSA,NB_BARRO VERMELHO,NB_BELA VISTA,NB_BENTO FERREIRA,NB_BOA VISTA,NB_BONFIM,NB_CARATOÍRA,NB_CENTRO,NB_COMDUSA,NB_CONQUISTA,NB_CONSOLAÇÃO,NB_CRUZAMENTO,NB_DA PENHA,NB_DE LOURDES,NB_DO CABRAL,NB_DO MOSCOSO,NB_DO QUADRO,NB_ENSEADA DO SUÁ,NB_ESTRELINHA,NB_FONTE GRANDE,NB_FORTE SÃO JOÃO,NB_FRADINHOS,NB_GOIABEIRAS,NB_GRANDE VITÓRIA,NB_GURIGICA,NB_HORTO,NB_ILHA DAS CAIEIRAS,NB_ILHA DE SANTA MARIA,...,NB_MONTE BELO,NB_MORADA DE CAMBURI,NB_MÁRIO CYPRESTE,NB_NAZARETH,NB_NOVA PALESTINA,NB_PARQUE INDUSTRIAL,NB_PARQUE MOSCOSO,NB_PIEDADE,NB_PONTAL DE CAMBURI,NB_PRAIA DO CANTO,NB_PRAIA DO SUÁ,NB_REDENÇÃO,NB_REPÚBLICA,NB_RESISTÊNCIA,NB_ROMÃO,NB_SANTA CECÍLIA,NB_SANTA CLARA,NB_SANTA HELENA,NB_SANTA LUÍZA,NB_SANTA LÚCIA,NB_SANTA MARTHA,NB_SANTA TEREZA,NB_SANTO ANDRÉ,NB_SANTO ANTÔNIO,NB_SANTOS DUMONT,NB_SANTOS REIS,NB_SEGURANÇA DO LAR,NB_SOLON BORGES,NB_SÃO BENEDITO,NB_SÃO CRISTÓVÃO,NB_SÃO JOSÉ,NB_SÃO PEDRO,NB_TABUAZEIRO,NB_UNIVERSITÁRIO,NB_VILA RUBIM,Hcap_0,Hcap_1,Hcap_2,Hcap_3,Hcap_4
0,0,62,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,56,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#Organize dataframe

##put the dependent variable to the front

In [24]:
col_name="No-show"
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)
df.head()

,No-show,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,daystoappoint,NB_AEROPORTO,NB_ANDORINHAS,NB_ANTÔNIO HONÓRIO,NB_ARIOVALDO FAVALESSA,NB_BARRO VERMELHO,NB_BELA VISTA,NB_BENTO FERREIRA,NB_BOA VISTA,NB_BONFIM,NB_CARATOÍRA,NB_CENTRO,NB_COMDUSA,NB_CONQUISTA,NB_CONSOLAÇÃO,NB_CRUZAMENTO,NB_DA PENHA,NB_DE LOURDES,NB_DO CABRAL,NB_DO MOSCOSO,NB_DO QUADRO,NB_ENSEADA DO SUÁ,NB_ESTRELINHA,NB_FONTE GRANDE,NB_FORTE SÃO JOÃO,NB_FRADINHOS,NB_GOIABEIRAS,NB_GRANDE VITÓRIA,NB_GURIGICA,NB_HORTO,NB_ILHA DAS CAIEIRAS,NB_ILHA DE SANTA MARIA,...,NB_MONTE BELO,NB_MORADA DE CAMBURI,NB_MÁRIO CYPRESTE,NB_NAZARETH,NB_NOVA PALESTINA,NB_PARQUE INDUSTRIAL,NB_PARQUE MOSCOSO,NB_PIEDADE,NB_PONTAL DE CAMBURI,NB_PRAIA DO CANTO,NB_PRAIA DO SUÁ,NB_REDENÇÃO,NB_REPÚBLICA,NB_RESISTÊNCIA,NB_ROMÃO,NB_SANTA CECÍLIA,NB_SANTA CLARA,NB_SANTA HELENA,NB_SANTA LUÍZA,NB_SANTA LÚCIA,NB_SANTA MARTHA,NB_SANTA TEREZA,NB_SANTO ANDRÉ,NB_SANTO ANTÔNIO,NB_SANTOS DUMONT,NB_SANTOS REIS,NB_SEGURANÇA DO LAR,NB_SOLON BORGES,NB_SÃO BENEDITO,NB_SÃO CRISTÓVÃO,NB_SÃO JOSÉ,NB_SÃO PEDRO,NB_TABUAZEIRO,NB_UNIVERSITÁRIO,NB_VILA RUBIM,Hcap_0,Hcap_1,Hcap_2,Hcap_3,Hcap_4
0,0,0,62,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,1,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,56,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [27]:
profile = ProfileReport( df, title='Pandas profiling report' , html={'style':{'full_width':True}})

profile.to_notebook_iframe()

TypeError: ignored